In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from env import api_key
import time

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [2]:
response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/battlebud?api_key={api_key}")

In [3]:
battlebud = response.json()

In [4]:
battlebud['puuid']

'8w5OLGrYy-YhUvX08ZD6a4sBvBvZuYTXU6UnfYhC1MBNo8rlV4OEODS9bNrqMBVuWirDBYOeMhWW0Q'

In [5]:
url = 'https://americas.api.riotgames.com/'

query = f'lol/match/v5/matches/by-puuid/8w5OLGrYy-YhUvX08ZD6a4sBvBvZuYTXU6UnfYhC1MBNo8rlV4OEODS9bNrqMBVuWirDBYOeMhWW0Q/ids?start=0&count=20&api_key={api_key}'

In [6]:
response = requests.get(url+query)

In [7]:
match_ids = response.json()

In [8]:
len(match_ids)

20

In [9]:
match_ids

['NA1_4083906028',
 'NA1_4083910411',
 'NA1_4083561526',
 'NA1_4069143161',
 'NA1_4069101224',
 'NA1_3967840881',
 'NA1_3967246118',
 'NA1_3966627103',
 'NA1_3966602533',
 'NA1_3966566502',
 'NA1_3966562962',
 'NA1_3966404211',
 'NA1_3966149816',
 'NA1_3965654430',
 'NA1_3965611323',
 'NA1_3965186467',
 'NA1_3965183441',
 'NA1_3964851235',
 'NA1_3964196435',
 'NA1_3963932851']

In [10]:
url = 'https://na.op.gg/ranking/ladder/'
response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
soup = BeautifulSoup(response.text)
table = soup.select('.ranking-table')
table = table[0]
table = soup.select('.ranking-table__cell')

In [11]:
table

[<td class="ranking-table__cell ranking-table__cell--rank">6</td>,
 <td class="ranking-table__cell ranking-table__cell--summoner"><a href="//na.op.gg/summoner/userName=Johnsun123"><img onerror="this.src='//opgg-static.akamaized.net/images/profile_icons/profileIcondefault.jpg';" src="//opgg-static.akamaized.net/images/profile_icons/profileIcon3785.jpg?image=c_scale,q_auto,c_scale,w_38&amp;v=1518361200"/><span>Johnsun123</span></a></td>,
 <td class="ranking-table__cell ranking-table__cell--tier">
 									Challenger
 								</td>,
 <td class="ranking-table__cell ranking-table__cell--lp">
 									1,517 LP
 								</td>,
 <td class="ranking-table__cell ranking-table__cell--level">
 								63
 							</td>,
 <td class="ranking-table__cell ranking-table__cell--winratio"><div class="winratio"><div class="winratio-graph"><div class="winratio-graph__fill winratio-graph__fill--left" style="width: 61%"></div><div class="winratio-graph__text winratio-graph__text--left">206</div><div class="w

In [12]:
def get_username(table):
    username_list = []
    for i in range (1,571,6):
        name = str(table[i])
        username = str(re.findall(r'userName=.+?"', name))
        username_list.append(username)
        df = pd.DataFrame(username_list)
        df = df.rename(columns={0:"pro"})
        df['pro'] = df.pro.str.replace("\['userName=","")
        df['pro'] = df.pro.str.replace("+",' ')
        df['pro'] = df.pro.str.replace("\"']",'')
        df = df[~df.pro.str.contains('%')]
        df = df.reset_index()
    return df

In [13]:
def op_gg_webscrape(pages):
    player_names = []
    for i in range(pages + 1):
        url = (f'https://na.op.gg/ranking/ladder/page={i}')
        response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
        soup = BeautifulSoup(response.text)
        table = soup.select('.ranking-table')
        table = table[0]
        table = soup.select('.ranking-table__cell')
        users_df = get_username(table)
        for pro in users_df.pro:
                player_names.append(str(pro))
    return player_names

In [14]:
player_list = op_gg_webscrape(10)

<ipython-input-12-c953d951acde>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\['userName=","")
<ipython-input-12-c953d951acde>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['pro'] = df.pro.str.replace("+",' ')
<ipython-input-12-c953d951acde>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\"']",'')


In [15]:
player_list = list(set(player_list))

In [16]:
len(player_list)

883

In [17]:
player_df = pd.DataFrame({'players':player_list})

In [18]:
player_df.to_csv('more_players.csv')

In [19]:
def op_gg_webscrape(pages):
    player_names = []
    url = 'https://na.op.gg/ranking/ladder/'
    response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
    soup = BeautifulSoup(response.text)
    table = soup.select('.ranking-table')
    table = table[0]
    table = soup.select('.ranking-table__cell')
    users_df = get_username(table)
    for pro in users_df.pro:
        player_names.append(pro)
        for i in range(pages + 1):
            url = (f'https://na.op.gg/ranking/ladder/page={i}')
            response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
            soup = BeautifulSoup(response.text)
            table = soup.select('.ranking-table')
            table = table[0]
            table = soup.select('.ranking-table__cell')
            users_df = get_username(table)
            for pro in users_df.pro:
                player_names.append(pro)
    return player_names

In [ ]:
player_list

In [ ]:
player_names = []
url = 'https://na.op.gg/ranking/ladder/'
response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
soup = BeautifulSoup(response.text)
table = soup.select('.ranking-table')
table = table[0]
names = soup.select('.ranking-table__cell')
for i in range (1,571,6):
    #summoner_names = re.match(r'userName=.*',str(names[i]))
    name = str(names[i])
    username = str(re.findall(r'userName=.+?"', name))
    player_names.append(username)

In [ ]:
Tçº

In [ ]:
df = pd.DataFrame(player_names)

In [ ]:
df = df.rename(columns={0:"pro"})

In [ ]:
df

In [ ]:
df['pro'] = df.pro.str.replace("\['userName=","")

In [ ]:
df['pro'] = df.pro.str.replace("+",' ')

In [ ]:
df['pro'] = df.pro.str.replace("\"']",'')

In [ ]:
df.shape

In [ ]:
df['pro'] = df.where(df.pro.str.contains('%') == False)

In [ ]:
df = df[~df.pro.str.contains('%')]

In [ ]:
df

In [ ]:
df.sample(10)

In [ ]:
len_list = []
for i in range (1,572,6):
    len_list.append(i)

In [ ]:
name = str(names[1])

In [ ]:
name

In [ ]:
thing = name[100:116]

In [ ]:
thing

In [ ]:
re.match(r'^("userName=")(.+?")$',name)

In [ ]:
username = str(re.findall(r'userName=.+?"', name))

In [ ]:
username

In [ ]:
username = username[11:-4]

In [ ]:
username

In [ ]:
username[-]

In [ ]:
username

In [ ]:
table[0]

In [6]:
df = pd.read_csv('more_players.csv')

,Unnamed: 0,players
0,0,Satoshi
1,1,Johnsun123
2,2,bogo binter
3,3,Sybr
4,4,good nightt
...,...,...
878,878,godsent mid
879,879,Alamander
880,880,hanason1
881,881,DabinPenguEmote


In [ ]:
df = pd.DataFrame(columns = ['puuid'])

In [ ]:
def get_puuid(name_list):
    puuid_list = []
    counter = 0
    for name in name_list:
        print(counter)
        counter += 1
        if counter%50 == 0:
            print('function paused')
            print
            time.sleep(180)
            print(puuid_list[counter - 5])
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info['puuid'])
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info['puuid'])
    return puuid_list

In [17]:
def get_puuid(name_list):
    puuid_list = []
    counter = 0
    for name in name_list:
        print(counter)
        counter += 1
        if counter%50 == 0:
            print('function paused')
            print
            time.sleep(180)
            print(puuid_list[counter - 5])
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info['puuid'])
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info['puuid'])
    return puuid_list

In [ ]:
puuid_list = get_puuid(df.players)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
function paused
rxhacqis2o6sQvSWCONLPvuw-1EZIAgtbLYEyglaYwbo67y5IaYMb6sBh97XfkOcaCtx2mYPvwuQPA
Continue
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
function paused
tm6B2ZX6h2Wf90xgbi_c_WDLO5-XlgJd6zpWTtOfhpX5Mr6Q687wDZG7dVJvbZtf3FddzLvPI2olvw
Continue
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
function paused
G23sTraHCOHar3LAZU07A7ShBw2h54PeuJ1xwR51P6Y7nVkBXhS2OeqczrobyRUje6twBq6MjLxs7A
Continue
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
19

In [16]:
puuid_list

NameError: name 'puuid_list' is not defined

In [ ]:
response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/battlebud?api_key={api_key}")

In [ ]:
summoner_info = response.json()

In [ ]:
battlebud